# Examination of efficient NHL roster design

## purpose of notebook:

- import all csv files from the 2010 season
- categorize and merge all csv files to create final data frame
- remove irrelavant observations
- keep the first two games of the season for analysis
- store final data frame

In [44]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## import data frames

- import play by play csv file

In [45]:
dm = pd.read_csv('t_play_by_play_o.csv')
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])
dm = dm.rename(columns={'VPlayer1Position': 'VPosition1', 'VPlayer2Position': 'VPosition2', 'VPlayer3Position': 'VPosition3', 'VPlayer4Position': 'VPosition4', 'VPlayer5Position': 'VPosition5', 'VPlayer6Position': 'VPosition6', 'HPlayer1Position': 'HPosition1', 'HPlayer2Position': 'HPosition2', 'HPlayer3Position': 'HPosition3', 'HPlayer4Position': 'HPosition4', 'HPlayer5Position': 'HPosition5', 'HPlayer6Position': 'HPosition6' })
dm = dm [['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType', 'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail', 'VPlayer1', 'VPosition1', 'VPlayer2', 'VPosition2', 'VPlayer3', 'VPosition3', 'VPlayer4', 'VPosition4', 'VPlayer5', 'VPosition5', 'VPlayer6', 'VPosition6', 'HPlayer1', 'HPosition1', 'HPlayer2', 'HPosition2', 'HPlayer3', 'HPosition3', 'HPlayer4', 'HPosition4', 'HPlayer5', 'HPosition5', 'HPlayer6', 'HPosition6']]

- import play by play giveaway csv file and rename columns

In [46]:
dg = pd.read_csv('t_play_by_play_giveaway_detail_o.csv')
dg = dg.rename(columns={'GivePlayerNumber': 'PlayerNumber', 'GivePlayerLName': 'PlayerName', 'GiveTeamCode': 'TeamCode'})

- import play by play takeaway csv file and rename columns

In [47]:
dt = pd.read_csv('t_play_by_play_takeaway_detail_o.csv')
dt = dt.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dt = dt.rename(columns={'TakePlayerNumber': 'PlayerNumber', 'TakePlayerLName': 'PlayerName', 'TakeTeamCode': 'TeamCode'})

- import goal detail csv file and rename column

In [48]:
dl = pd.read_csv('t_play_by_play_goal_detail_o.csv')
dl = dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dl = dl.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play shot detail csv file and rename column

In [49]:
ds = pd.read_csv('t_play_by_play_shot_detail_o.csv')
ds = ds.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
ds = ds.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play miss detail csv file and rename column

In [50]:
dn = pd.read_csv('t_play_by_play_miss_detail_o.csv')
dn = dn.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dn = dn.rename(columns={'PlayerLName': 'PlayerName'})

- import play by play block detail csv file and rename columns

In [51]:
db = pd.read_csv('t_play_by_play_block_detail_o.csv')
db = db.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
db = db.rename(columns={'ShotPlayerLName': 'ShotPlayerName', 'BlockPlayerLName': 'PlayerName', 'BlockPlayerNumber': 'PlayerNumber', 'BlockTeamCode': 'TeamCode'})

- import play by play hit detail csv file and rename columns

In [52]:
dh = pd.read_csv('t_play_by_play_hit_detail_o.csv')
dh = dh.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dh = dh.rename(columns={'HitterPlayerLName': 'HitterPlayerName', 'HitteePlayerLName': 'HitteePlayerName'})

 - import play by play faceoff detail csv file and rename columns

In [53]:
df = pd.read_csv('t_play_by_play_faceoff_detail_o.csv')
df = df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
df = df.rename(columns={'VPlayerLName': 'VName', 'HPlayerLName': 'HName', 'VPlayerNumber': 'VNumber', 'HPlayerNumber': 'HNumber'})

 - import play by play penalty detail csv file and rename columns

In [54]:
dp = pd.read_csv('t_play_by_play_penalty_detail_o.csv')
dp = dp.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dp = dp.rename(columns={'PenaltyPlayerLName': 'PenaltyPlayerName', 'DrawnByPlayerLName': 'DrawnByPlayerName'})

- import game detail csv file 

In [55]:
dd = pd.read_csv('t_game_detail_o.csv')

- import scoring csv file and rename columns

In [56]:
dc = pd.read_csv('t_scoring_detail_o.csv')
dc['GoalTime'] = dc['Time'].copy()
dc = dc.rename(columns={'Time': 'EventTimeFromZero', 'GoalType': 'AdvantageType'})
dc = dc [['Season', 'GameNumber', 'GoalNumber', 'Period', 'GoalTime', 'EventTimeFromZero', 'AdvantageType', 'TeamCode', 'Assist1Player', 'Assist2Player']]

## group and merge data frames based on similarity

Since giveaway (dg) and takeaway (dt) csv files have identical columns, they are merged together. Thus, dt is merged on dg:

## merge giveaways and takeaways 

In [57]:
dg = dg.merge(dt, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dg.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

dg now contains observations from both type of events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge goal,  shot,  miss and block dataframes

Goal, miss and block are result of a shot. For that reason, they are merged together. 

- merge shot data frame (ds) onto goal data frame (dl):

In [58]:
dl = dl.merge(ds, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from both shot and goal data set. The merged data frame is ascendingly sorted by season, game number and event number.

- merge miss dataframe (dm) onto goal data frame (dl): 

In [59]:
dl = dl.merge(dn, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**dl** now contains observations from goal, shot and miss events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge block (db) onto goal dataframe (dl):

In [60]:
dl = dl.merge(db, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**dl** now contains observations from goal, shot, miss and block events. The merged data frame is ascendingly sorted by season, game number and event number.

In [61]:
dl.columns

Index(['GameNumber', 'EventNumber', 'TeamCode', 'PlayerNumber', 'PlayerName',
       'ShotType', 'Zone', 'Length', 'Season', 'ShotResult', 'ShotTeamCode',
       'ShotPlayerNumber', 'ShotPlayerName'],
      dtype='object')

## merge faceoff, hit and penalty together

- merge hit data (dh) frame onto faceoff data frame (df):

In [62]:
df = df.merge(dh, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**df** now contains observations from both faceoff and hit events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge penalty data frame (dh) onto faceoff data frame (df):

In [63]:
df = df.merge(dp, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

**df** now contains observations from faceoff, hit and penalty events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge merged data frames

Three merged data frames have been created:  dg, dl and df. The next step is to merge the merged data frames together.

- merge dg onto dl:

In [64]:
dl = dl.merge(dg, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from goal, shot, miss , block, giveaway and takeaway events. The merged data frame is ascendingly sorted by season, game number and event number.

- merge df onto dl:

In [65]:
dl = dl.merge(df, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


**dl** now contains observations from goal, shot, miss , block, giveaway, takeaway, faceoff, hit and penalty events. The merged data frame is ascendingly sorted by season, game number and event number.

## merge detail play (dd) onto all ice events (dl)

In [66]:
dl = dl.merge(dd, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [67]:
dl.columns

Index(['GameNumber', 'EventNumber', 'TeamCode', 'PlayerNumber', 'PlayerName',
       'ShotType', 'Zone', 'Length', 'Season', 'ShotResult', 'ShotTeamCode',
       'ShotPlayerNumber', 'ShotPlayerName', 'WinTeamCode', 'VTeamCode',
       'VNumber', 'VName', 'HTeamCode', 'HNumber', 'HName', 'HitterTeamCode',
       'HitterPlayerNumber', 'HitterPlayerName', 'HitteeTeamCode',
       'HitteePlayerNumber', 'HitteePlayerName', 'PenaltyTeamCode',
       'PenaltyPlayerNumber', 'PenaltyPlayerName', 'PenaltyType',
       'DrawnByTeamCode', 'DrawnByPlayerNumber', 'DrawnByPlayerName',
       'GameDate'],
      dtype='object')

## merge all on-ice events (dl) onto play by play (dm)

The merged data frame containing all on-ice events (dl) is merged onto the play by play data frame by season, game number and event number.

In [68]:
dm = dm.merge(dl, on=['Season', 'GameNumber', 'EventNumber'], how='outer')
dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

## merge scoring detail (dc) onto play by play (dm)

The final merge is to add scoring detail data frame (dc) onto play by play data frame (dm). The purpose is to include assist 1 player name, assist 1 player number, assist 2 player number and assist player 2 name into the data frame that will be used for player evaluation analysis.

In [69]:
dm = dm.merge(dc, on=['Season', 'GameNumber', 'Period', 'EventTimeFromZero', 'AdvantageType', 'TeamCode'], how='outer')

## remove irrelevant observations


Tv time-out, goalie time-out and icing are listed as "stoppage" events and are removed from the data frame as they have no impact on the probability of a goal being scored.

In [70]:
dm = dm[dm['EventType']!='STOP']

## man-advantage scenarios

In [71]:
value_list = ['PP', 'SH']
dm[dm['AdvantageType'].isin(value_list)]
dm = dm[dm['AdvantageType'] != 'PP']
dm = dm[dm['AdvantageType'] != 'SH']
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')

Since the player evaluation model uses only even strength situations, man-advantage scenarios are dropped from the data frame

## keep first two games of the season

To conduct analysis for both player evalution model and roster design model, the first two game of the 2010 season are used. 

In [72]:
dm = dm[dm['GameNumber'] <= 20002]

# store final data frame

The merged play by play data frame is stored 

In [73]:
dm.to_csv('pbpmerge.csv', index='False', sep=',')

The next step is to conduct the player evaluation model, which will take place in the "roster_design_player_evalution" notebook.